In [1]:
import pandas as pd
import numpy as np
import os
import heapq

In [49]:
class Pre_Processer:
    def __init__(self):
        self.size = 150*10**3
        self.size_name = "medium"
        
    def read_data(self):
        self.data_users = pd.read_excel("Data.xlsx", sheet_name="Users")
        self.data_tracking = pd.read_excel("Data.xlsx", sheet_name="Tracking")
        
    def read_processed_data(self):
        # Read_Data: load user data from csv.
        self.user_data = pd.read_csv(f"df_user_data_{self.size_name}.csv")
        self.user_data = self.user_data.drop("Unnamed: 0", axis=1)
        
    def read_df_ratings_matrix(self):
        self.matrix_asFrame = pd.read_csv(f"df_ratings_matrix_{self.size_name}.csv")
        
    def pre_processing(self):
        print("You are starting with Pre-Processing step 1 here. Otherwise run read_processed_data()")
        
        self.data_tracking = self.data_tracking.drop(labels=[' accessType;;;;;', 'Unnamed: 9',
        'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13',
        'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17',
        'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21',
        'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25',
        'Unnamed: 26', 'Unnamed: 27', 'Unnamed: 28', 'Unnamed: 29',
        'Unnamed: 30', 'Unnamed: 31', 'Unnamed: 32', 'Unnamed: 33'], axis=1)
        
        self.data_tracking.columns = ['idUser', 'day', 'month', 'year', 'idContent', 'contentSubType',
               'contentBranch', 'contentName']
    
    def set_ratings_size(self, exponent):
        self.size = 150*10**exponent
        
        if exponent == 1:
            self.size_name = "extra_small"
        elif exponent == 2:
            self.size_name = "small"
        elif exponent == 3:
            self.size_name = "medium"
        elif exponent == 4:
            self.size_name = "large"
        elif exponent == 5:
            self.size_name = "extra_large"
        else:
            self.size_name = "different_size"
            
        print(f"Data Frame size set to: {self.size_name}.")
            
        
    def create_ratings_data(self):
        # v2.0 Get all the viewed content for each user
        user_data = pd.DataFrame(columns=["idUser", "idContent", "timesViewed"])
        data_dict = {"idUser": "user", "idContent": "content", "timesViewed": "viewed"}
        user = []
        content = []
        viewed = []

        for i in range(0, len(self.data_tracking["idUser"])):
            if i == self.size:
                break

            # Save & Create new user_list if this row starts a new users data.
            if i != 0 and self.data_tracking["idUser"].iloc[i] != self.data_tracking["idUser"].iloc[i-1]:
                data_dict = {"idUser": user, "idContent": content, "timesViewed": viewed}
                this_user_data = pd.DataFrame.from_dict(data_dict)
                user_data = pd.concat([user_data, this_user_data])
                user = []
                content = []
                viewed = []

            item_found = False

            for j in range(len(content), 0, -1):
                if content[j-1] == self.data_tracking["idContent"].iloc[i]:
                    viewed[j-1] += 1  
                    item_found = True
                    break

            if len(content) == 0 or item_found == False:
                user.append(self.data_tracking["idUser"].iloc[i]) 
                content.append(self.data_tracking["idContent"].iloc[i]) 
                viewed.append(1)

        # save the user data frame to save time
        user_data.to_csv(f"df_user_data_{self.size_name}.csv")   
        
    def create_ratings_matrix(self):
        print("You are starting with Pre-Processing step 2 here. Otherwise run pre_processing()")
        
        num_contents = 0
        num_users = 0
        id_contents = list()
        id_users = list()
        
        for id in Data.user_data["idContent"]:
            if id not in id_contents:
                id_contents.append(id)
                num_contents += 1
                
        for id in Data.user_data["idUser"]:
            if id not in id_users:
                id_users.append(id)
                num_users += 1
                
        self.matrix_asFrame = pd.DataFrame(index=id_users, columns=id_contents)
        
        # create unary matrix
        """
        for user in self.matrix_asFrame.index:
            for content in self.user_data[self.user_data["idUser"] == user]["idContent"]:
                    self.matrix_asFrame.loc[user, content] = 1"""
        
        # create numerical matrix with view counts
        for user in self.matrix_asFrame.index:
            for j in range(0, len(self.user_data[self.user_data["idUser"] == user]["idContent"])):
                    content = self.user_data[self.user_data["idUser"] == user].iloc[j]["idContent"]
                    views = self.user_data[self.user_data["idUser"] == user].iloc[j]["timesViewed"]
                    self.matrix_asFrame.loc[user, content] = views

        self.matrix_asFrame = self.matrix_asFrame.fillna(0)
        
        # save the user data frame to save time
        self.matrix_asFrame.to_csv(f"df_ratings_matrix_{self.size_name}.csv") 
        
    def create_ratings_matrix(self):
        print("You are starting with Pre-Processing step 3 here. Otherwise run pre_processing()")
        
        self.ratings_matrix = self.matrix_asFrame.to_numpy()
        np.save("ratings_matrix", self.ratings_matrix)

In [50]:
Data = Pre_Processer()

In [ ]:
# Full pre-processing step 1. Optional: set_ratings_size(). If not set 150*10**3 = 150.000 rows of tracking data will be read.
Data.read_data()
Data.pre_processing()
Data.set_ratings_size(2)
Data.create_ratings_data()

In [39]:
# If Pre-Processing Step 1 is already done.
Data.set_ratings_size(2)
Data.read_processed_data()
Data.create_ratings_matrix()

Data Frame size set to: small.
You are starting with Pre-Processing step 2 here. Otherwise run pre_processing()


In [51]:
# If Pre-Processing Step 1 & 2 are already done.
Data.set_ratings_size(2)
Data.read_df_ratings_matrix()
Data.create_ratings_matrix()

Data Frame size set to: small.
You are starting with Pre-Processing step 3 here. Otherwise run pre_processing()


In [40]:
Data.matrix_asFrame.head(10)

,175415,170458,182197,163274,169625,168536,163193,163191,175430,175419,...,308383,300411,214143,220197,203375,244940,168064,301925,262676,24350
17208,16,12,1,1,2,1,2,2,2,1,...,0,0,0,0,0,0,0,0,0,0
20490,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25554,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26940,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29023,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
31451,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
33139,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
38156,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
39646,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
44362,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
